# Nodding Data Reduction
------------------------

This notebook shows the reduction path of a nodding observation in a multi-beam receiver.

Similar to a position-switched observation, two beams are selected for simultaneous observing (though the receiver can have more than two beams). In the first scan BEAM1 is looking at the source, while BEAM2 is looking at an assumed OFF position. In the next scan, BEAM2 will be looking at the source, while BEAM1 is looking at (another) OFF position. This will result in two position-switched solutions, which are then averaged for the final spectrum. 

The obvious advantage of this observing mode is that the telescope is always ON source, bringing the noise down by $\sqrt{2}$ compared to a classic position switched observation. Minus a small amount of slewing time of course. However, the beam separation in the receiver should be large enough to ensure there are good OFF positions, otherwise a proper Position Switching observation is needed with a large enough offset.

For fun, this notebook starts with  a matching getps() observation from another scan so we can confirm the improvement in signal/noise.

The data in this notebook were also presented in https://gbtdocs.readthedocs.io/en/latest/how-tos/data_reduction/gbtidl.html#basic-nodding in a similar GBTIDL data reduction.

## Background

The spectral line observed here is the NH$_3$ (1,1) line at 23.69 GHz with the K-band focal plane array (KFPA) receiver. This receiver has 7 beams: one central beam and six in a hexagonal (?) pattern. The source is a position in the W3 cloud.


In [ ]:
# %matplotlib widget

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from astropy import units as u
from dysh.fits.gbtfitsload import GBTFITSLoad
from dysh.util.files import dysh_data
from pathlib import Path
from dysh.util.download import from_url

Future use of dysh_data:

 filename = dysh_data(example="nod-KFPA/data/TGBT22A_503_02.raw.vegas")
 
 filename = dysh_data(example="nod-KFPA/data/TGBT22A_503_02.raw.vegas.trim.fits")

In [ ]:
url = "http://www.gb.nrao.edu/dysh/example_data/nod-KFPA/data/TGBT22A_503_02.raw.vegas.trim.fits"
savepath = Path.cwd() / "data"
filename = from_url(url, savepath)

This trimmed dataset is an extraction from a much larger dataset (19GB) and can take some time to load if it's the first time.

In [ ]:
sdf = GBTFITSLoad(filename)

In [ ]:
sdf.summary()

We will use scans 60 and 61 for the position-switching (PS) observation.

Also note for the 7-beam KFPA receiver, the central beam (fdnum=0) will be the source tracking beam for the Position Switch observation.

In [ ]:
sp0 = sdf.getps(scan=60,plnum=0,ifnum=0,fdnum=0).timeaverage()[1500:-1500]

In [ ]:
sp0s = sp0.smooth('box',51)
sp0s.plot(title='dysh getps',xaxis_unit="chan")
sp0s.stats()

Now the nodding method, using scans 62 and 63.  This will return one scanblock with two NodScan's, each representing a "PS" observations that need to be averaged to get the final spectrum.  We can do that with the timeaverage() function directly applying to this list of (two) NodScan's.  And for good measure we will smooth it a little bit as we did with the previous PS observation.

Normally the FITS header will contain information on which is BEAM1 and which is BEAM2. In this case beams 2 and 6 are the two nodding beams. For convenience we are dropped the first and last 1500 (of the 32768) channels of the spectrum to avoid edge effects. We leave it to the reader to view these beautiful edges.

In [ ]:
sp2 = sdf.getnod(scan=62,ifnum=0,plnum=0).timeaverage()[1500:-1500]
sp2s = sp2.smooth('box',51)
sp2s.stats()

In [ ]:
sp2s.plot(title='dysh getnod',xaxis_unit="chan")
print('getps  RMS:', sp0s[:200].stats()['rms'])
print('getnod RMS:', sp2s[:200].stats()['rms'])

The improvement of the RMS is not great, the 60 mK signal from the PS should have predicted more like 42 mK. In addition, the NOD spectrum shows the 4 side-peaks to be roughly the same, where the PS spectrum shows the outer two to be well below the middle two. Who is correct?

## Baseline Subtracted Spectra

Let us subtract a baseline and compare the two spectra on the same scale.

In [ ]:
sp0s.baseline(model="cheby", degree=2, exclude=[200,400], remove=True)
sp2s.baseline(model="cheby", degree=2, exclude=[200,400], remove=True)

In [ ]:
sp0s.plot(title='dysh getps',xaxis_unit="km/s", ymin=-0.2, ymax=1.0)
sp2s.plot(title='dysh getnod',xaxis_unit="km/s", ymin=-0.2, ymax=1.0)

In [ ]:
print("PS  spectrum peak:", sp0s.stats()["max"])
print("NOD spectrum peak:", sp2s.stats()["max"])

Given that the RMS is about 0.060 K, the peaks of these two spectra agree well.
